In [1]:
pip install fpdf pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from fpdf import FPDF
import os

# 1. CÀRREGA DE DADES
file_path = 'MUESTRA_DATASETFINAL.csv'
df = pd.read_csv(file_path, sep=';')

# Neteja de formats (comes a punts)
def clean_float(val):
    if isinstance(val, str):
        val = val.replace(',', '.')
    try:
        return float(val)
    except (ValueError, TypeError):
        return 0.0

df['CONSUMO_REAL_NUM'] = df['CONSUMO_REAL'].apply(clean_float)
# Important: Convertir la data correctament per poder ordenar
df['FECHA_HORA'] = pd.to_datetime(df['FECHA_HORA'], dayfirst=True, errors='coerce')

print("✅ Dades carregades.")

✅ Dades carregades.


In [22]:
# 2. SELECCIÓ DE CLIENT AMB DADES REALS
# Busquem el client que tingui més registres per poder fer una gràfica amb "cara i ulls"
top_client_id = df['POLISSA_SUBM'].value_counts().idxmax()
count_registres = df['POLISSA_SUBM'].value_counts().max()

print(f"Client seleccionat (el que té més dades): {top_client_id}")
print(f"Número de registres trobats a la BBDD: {count_registres}")

# Extraiem TOTES les files d'aquest client i les ordenem per data
# Això construeix l'històric real
historic_client = df[df['POLISSA_SUBM'] == top_client_id].sort_values('FECHA_HORA')

# Agafem les llistes reals per a la gràfica
dates_reals = historic_client['FECHA_HORA'].tolist()
consums_reals = historic_client['CONSUMO_REAL_NUM'].tolist()

# Dades estàtiques (agafem l'última fila per tenir la info més recent)
info_client = historic_client.iloc[-1]
consum_total_periode = historic_client['CONSUMO_REAL_NUM'].sum()

print("✅ Dades extretes de la BBDD (Sense invents).")

Client seleccionat (el que té més dades): QEPJ3GL36LPH6JMU
Número de registres trobats a la BBDD: 770
✅ Dades extretes de la BBDD (Sense invents).


In [23]:
# --- GRÀFICA 1: CONSUM REAL (BBDD) ---
plt.figure(figsize=(10, 5))

# Pintem les dades reals
plt.plot(dates_reals, consums_reals, marker='o', linestyle='-', color='#0072BC', linewidth=2, label='Consum Registrat')
plt.fill_between(dates_reals, consums_reals, color='#0072BC', alpha=0.1)

plt.title(f'Històric de Consum Real - {top_client_id}', fontsize=12)
plt.ylabel('Consum ($m^3$)', fontsize=10)
plt.grid(True, linestyle=':', alpha=0.6)

# Formategem l'eix X perquè les dates es llegeixin bé
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%Y'))
plt.gcf().autofmt_xdate() # Gira les dates si s'encavalquen

chart_consum_real = "chart_consum_real.png"
plt.savefig(chart_consum_real, dpi=100, bbox_inches='tight')
plt.close()


# --- GRÀFICA 2: PROBABILITAT (Model IA) ---
# Aquesta part normalment ve del model de predicció (no de la BBDD històrica)
import random
prob_avui = 0.88 # Simulació del model
days = ['Avui', 'Demà', '7 dies']
probs = [prob_avui, prob_avui*0.9, prob_avui*0.2]

plt.figure(figsize=(8, 4))
plt.plot(days, probs, marker='o', linestyle='--', color='#E65100', linewidth=2)
plt.title('Model Predictiu de Risc (GesAI)', fontsize=12)
plt.ylim(0, 1.1)
plt.grid(True, linestyle=':', alpha=0.6)

chart_prob = "chart_prob.png"
plt.savefig(chart_prob, dpi=100, bbox_inches='tight')
plt.close()

print("✅ Gràfiques generades.")

✅ Gràfiques generades.


In [24]:
class PDF_GesAI_Real(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 18)
        self.set_text_color(0, 114, 188)
        self.cell(0, 10, 'GesAI', 0, 1, 'L')
        self.set_font('Arial', '', 10)
        self.set_text_color(100, 100, 100)
        self.cell(0, 0, 'Informe Tècnic d\'Incidència', 0, 1, 'L')
        self.ln(8)
        self.set_draw_color(200, 200, 200)
        self.line(10, 22, 200, 22)
        self.ln(5)

    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.set_text_color(128, 128, 128)
        self.cell(0, 10, 'GesAI Project | Dades reals extretes del CRM', 0, 0, 'C')

    def section_title(self, title):
        self.set_font('Arial', 'B', 11)
        self.set_text_color(255, 255, 255)
        self.set_fill_color(0, 114, 188)
        self.cell(0, 7, f"  {title}", 0, 1, 'L', 1)
        self.ln(3)

    def data_pair(self, label, value, unit=""):
        self.set_font('Arial', 'B', 9)
        self.set_text_color(50, 50, 50)
        self.cell(50, 6, f"{label}:", 0, 0)
        
        if pd.isna(value) or value == "" or value == "No disponible":
            self.set_font('Arial', 'I', 9)
            self.set_text_color(160, 160, 160)
            display_text = "Dada no disponible"
        else:
            self.set_font('Arial', '', 9)
            self.set_text_color(0, 0, 0)
            display_text = f"{value} {unit}".strip()

        self.cell(0, 6, display_text, 0, 1)

def create_real_report(row, total_consum, d_reals, path_prob, path_consum):
    pdf = PDF_GesAI_Real()
    pdf.add_page()

    # 1. IDENTIFICACIÓ
    pdf.section_title("1. IDENTIFICACIÓ I CONTACTE")
    pdf.data_pair("Titular Pòlissa", row['POLISSA_SUBM'])
    pdf.data_pair("Comptador", row['NUMEROSERIECONTADOR'])
    pdf.data_pair("Ubicació", row['SECCIO_CENSAL'])
    pdf.data_pair("Telèfon", pd.NA) # NaN Guapo
    pdf.ln(4)

    # 2. DIAGNÒSTIC (Probabilitat)
    pdf.section_title("2. DIAGNÒSTIC PREDICTIU (IA)")
    pdf.ln(2)
    pdf.image(path_prob, x=20, w=160)
    pdf.ln(5)

    # 3. CONSUM REAL
    pdf.section_title("3. EVIDÈNCIA DE CONSUM (DADES BBDD)")
    pdf.ln(2)
    
    # Dades reals
    registres = len(d_reals)
    data_ini = d_reals[0].strftime('%d/%m/%Y')
    data_fi = d_reals[-1].strftime('%d/%m/%Y')
    
    pdf.data_pair("Període mostrat", f"{data_ini} - {data_fi}")
    pdf.data_pair("Nº Lectures", str(registres))
    pdf.data_pair("Consum Acumulat", f"{total_consum:.2f}", "m³")
    pdf.ln(2)
    
    # Gràfica Real
    pdf.image(path_consum, x=20, w=160)
    pdf.ln(2)
    pdf.set_font('Arial', 'I', 8)
    pdf.cell(0, 5, "Fig 2. Evolució real del consum segons dades de facturació/lectura.", 0, 1, 'C')
    
    filename = f"Informe_Real_{row['POLISSA_SUBM']}.pdf"
    pdf.output(filename)
    return filename

# GENERAR PDF
pdf_final = create_real_report(info_client, consum_total_periode, dates_reals, chart_prob, chart_consum_real)
print(f"🎉 PDF generat amb dades reals: {pdf_final}")

🎉 PDF generat amb dades reals: Informe_Real_QEPJ3GL36LPH6JMU.pdf
